Importar a biblioteca

In [ ]:
from moviepy.editor import VideoFileClip, concatenate_videoclips, vfx, AudioFileClip
import numpy as np

Definir o caminho do vídeo original

In [ ]:
video = VideoFileClip("input_video.mp4")

Inverter a imagem no eixo a cada 20 segundos

In [ ]:
video_clips = []


segment_duration = 20  
volume_intervals = [0, 30, 60, 90, 120, video_duration - 10]
volume_levels = [1.0, 0.5, 0.25, 0.125, 0.0625, 0.0]

num_segments = int(np.ceil(video_duration / segment_duration))

for i in range(num_segments):
    start_time = i * segment_duration
    end_time = min((i + 1) * segment_duration, video_duration)
    clip = video.subclip(start_time, end_time)
    
    if i % 2 == 0:
        clip = clip.fx(vfx.mirror_x)
    
    video_clips.append(clip)

processed_video = concatenate_videoclips(video_clips)

Diminuir o som gradativamente

In [ ]:
audio = video.audio

def adjust_volume(get_frame, t):
    frame = get_frame(t)
    t = np.array(t) 
    volume = np.zeros_like(t, dtype=float)

    for i in range(len(volume_intervals) - 1):
        mask = (volume_intervals[i] <= t) & (t < volume_intervals[i + 1])
        volume[mask] = volume_levels[i]

    mask = t >= volume_intervals[-1]
    volume[mask] = volume_levels[-1]

    if frame.ndim == 1:
        return volume * frame
    else:
        return (volume[:, np.newaxis]) * frame


adjusted_audio = audio.fl(adjust_volume)

processed_video = processed_video.set_audio(adjusted_audio)

Cortar os segundos 60 a 80 do vídeo

In [ ]:
clip_before_60 = processed_video.subclip(0, 60)

clip_after_80 = processed_video.subclip(80, processed_video.duration)

clip_60_to_80 = processed_video.subclip(60, 80)

final_video = concatenate_videoclips([clip_before_60, clip_after_80, clip_60_to_80])

Final

In [ ]:
final_video.write_videofile("output_video.mp4", codec="libx264", audio_codec="aac")